<a href="https://colab.research.google.com/github/GabrielFCosta/TCC-BSI-2023.1/blob/main/old/matc97_gabriel_teste_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import string
import nltk
import math
import pandas as pd
from urllib.request import urlopen
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [ ]:
# Remove pontuação, utilizada pela função seguinte de pré-processamento
def remover_pontuacao(tokens):
  for token in tokens:
    if token in string.punctuation:
      tokens.remove(token)
# Função de pré-processamento
def preprocess(text):
    text = word_tokenize(text)
    remover_pontuacao(text)
    return pos_tag(text)

# Listas para coletar rótulos e entidades (palavras) do texto
codes = []
words = []
def  clearlists():
  if len(words) > 0:
    words.clear()
  if len(codes) > 0:
    codes.clear()

# Função de reconhecimento de entidades nomeadas (NER)
def ner(sent):
  ne_tree = nltk.ne_chunk(sent)
  for chunk in ne_tree:
    if hasattr(chunk,"label"):
      word =' '.join(c[0] for c in chunk)
     #print(chunk.label(),word)
      codes.append(chunk.label())
      words.append(word)

# Função que retorna dataframe de entidades (palavras), rótulos e frequências 
def returndataset(col1,col2):
  # junta listas de palavras e rótulos num dataframe
  dataset = pd.DataFrame(zip(words,codes),columns =[col1,col2])
  clearlists()
  # calcula frequências das palavras e adiciona nova coluna
  dataset['freq'] = dataset.groupby([col1,col2])[col1].transform('count')
  return dataclean(dataset)
 
# Remove linhas duplicatas e atualiza index
def dataclean(dataset):
  dataset.drop_duplicates(inplace= True)
  dataset.reset_index()
  return dataset

# Verifica rótulo gramatical
def test(tag):
  # verificando todos substantivos, adjetivos, verbos e numerais cardinais.
  tags = ['NN','NNS','NNP','NNPS','JJ','JJR','JJS','VB','VBD','VBG','VBN','VBP','VBZ','CD']
  #tags = ['NN','NNS','NNP','NNPS']
  for i in tags:
    if i == tag:
      return True
  return False

# Filtra palavras da lista gerada pelo pos tagging e retorna dataframe
def filtertuples(textlist,rtdata):
  for tuple in textlist:
    if test(tuple[1]):
      words.append(tuple[0])
      codes.append(tuple[1])
  if rtdata:
    return returndataset('word','pos_tag')
  else:
    aux = words.copy()
    clearlists()
  return aux

# Retorna união de dois dataframes com "frequência de documentos" 
def datamerge(words0,words1,col1,col2):
  merged = pd.merge(words0,words1, on=[col1,col2], how='outer')
  merged = dataclean(merged).fillna(0)
  doc_freq = []
  c = 0
  # verifica duas colunas de frequência
  for i in merged['freq_x'].tolist():
    # se frequência da palavras maior que 0 em ambos documentos 
    if i > 0 and merged['freq_y'].tolist()[c] > 0:
      doc_freq.append(2)
    else:
      doc_freq.append(1)
    c += 1
  merged['doc_freq'] = doc_freq
  return merged.sort_values(by=[col1,col2])

def jaccard_simples(doc_freq):
    intersecao = 0
    c = 0
    for x in doc_freq:
      if x == 2:
        intersecao += 1
      c += 1
    print("interseção:",intersecao)
    uniao = len(doc_freq)
    print("união:",uniao)
    print("jaccard simplificada:",intersecao/uniao)

In [ ]:
# Execução do reconhecimento de entidades nomeadadas:

#urls = ["https://raw.githubusercontent.com/GabrielFCosta/preprocessamento/main/noticia.txt",
#        "https://raw.githubusercontent.com/GabrielFCosta/preprocessamento/main/noticia1.txt",
#        "https://raw.githubusercontent.com/GabrielFCosta/preprocessamento/main/noticia2.txt"]
# Lista que vai conter dataframes de entidades (corpus)
datasets = []

urls = ["Airbnb will shut down its listings in China after two years of lockdowns in the country Airbnb will shut down its listings in China",
        "Starting this summer, Airbnb will take down its listings and offers for hosted experiences in China.",
        "International brands, from Apple to Estee Lauder, have warned of the financial impact of the restrictions."]
# Pra cada url
for url in urls:
  #data = urlopen(url)
  #ex = data.read().decode('utf-8')
  ex=nltk.sent_tokenize(url)
  print(ex)
  # Detectando entidades de nomeadas por sentença
  for sentence in ex:
    # Sentença Pré-processada
    text = preprocess(sentence)
    print(text)
    # Entidades Nomeadas na Sentença
    ner(text)
  # Adiciona dataframe de entidades nomeadas do texto + labels, frequências e TF
  datasets.append(returndataset('entity','label'))
c = 0
for dataset in datasets:
  print("\nstring:",c)
  print(dataset)
  c += 1

['Airbnb will shut down its listings in China after two years of lockdowns in the country Airbnb will shut down its listings in China']
[('Airbnb', 'NNP'), ('will', 'MD'), ('shut', 'VB'), ('down', 'RP'), ('its', 'PRP$'), ('listings', 'NNS'), ('in', 'IN'), ('China', 'NNP'), ('after', 'IN'), ('two', 'CD'), ('years', 'NNS'), ('of', 'IN'), ('lockdowns', 'NNS'), ('in', 'IN'), ('the', 'DT'), ('country', 'NN'), ('Airbnb', 'NNP'), ('will', 'MD'), ('shut', 'VB'), ('down', 'RP'), ('its', 'PRP$'), ('listings', 'NNS'), ('in', 'IN'), ('China', 'NNP')]
['Starting this summer, Airbnb will take down its listings and offers for hosted experiences in China.']
[('Starting', 'VBG'), ('this', 'DT'), ('summer', 'NN'), ('Airbnb', 'NNP'), ('will', 'MD'), ('take', 'VB'), ('down', 'RP'), ('its', 'PRP$'), ('listings', 'NNS'), ('and', 'CC'), ('offers', 'NNS'), ('for', 'IN'), ('hosted', 'JJ'), ('experiences', 'NNS'), ('in', 'IN'), ('China', 'NNP')]
['International brands, from Apple to Estee Lauder, have warned of

In [ ]:
print("\nComparando documento 0 consigo mesmo (caso base):")
dt = datamerge(datasets[0],datasets[0],'entity','label')
print(dt)
jaccard_simples(dt['doc_freq'].tolist())
print("\nComparando documentos 0 e 1:")
dt = datamerge(datasets[0],datasets[1],'entity','label')
print(dt)
jaccard_simples(dt['doc_freq'].tolist())
print("\nComparando documentos 0 e 2:")
dt = datamerge(datasets[0],datasets[2],'entity','label')
print(dt)
jaccard_simples(dt['doc_freq'].tolist())


Comparando documento 0 consigo mesmo (caso base):
   entity   label  freq_x  freq_y  doc_freq
0  Airbnb     GPE       1       1         2
2  Airbnb  PERSON       1       1         2
1   China     GPE       2       2         2
interseção: 3
união: 3
jaccard simplificada: 1.0

Comparando documentos 0 e 1:
   entity   label  freq_x  freq_y  doc_freq
0  Airbnb     GPE       1     0.0         1
2  Airbnb  PERSON       1     1.0         2
1   China     GPE       2     1.0         2
interseção: 2
união: 3
jaccard simplificada: 0.6666666666666666

Comparando documentos 0 e 2:
         entity   label  freq_x  freq_y  doc_freq
0        Airbnb     GPE     1.0     0.0         1
2        Airbnb  PERSON     1.0     0.0         1
3         Apple     GPE     0.0     1.0         1
1         China     GPE     2.0     0.0         1
4  Estee Lauder  PERSON     0.0     1.0         1
interseção: 0
união: 5
jaccard simplificada: 0.0


In [ ]:
def similaridade_WP_means(str0,str1):
  means = []
  for wd0 in str0:
    syn0 = wn.synsets(wd0)
    # verifica se synset0 contém alguma definição
    if len(syn0) > 0:
      val = 0
      for wd1 in str1:
        syn1 = wn.synsets(wd1)
        # verifica se synset1 contém alguma definição
        if len(syn1) > 0:
          val+= syn0[0].wup_similarity(syn1[0])
          #print("syn0:",syn0[0],"syn1:",syn1[0],"val:",val)
      means.append(val/len(str1))
  print(means)
  print(sum(means)/len(means))

def similaridade_WP_max(str0,str1):
  means = []
  vals = []
  aux = 0
  for wd0 in str0:
    syn0 = wn.synsets(wd0)
    # verifica se 1a palavra contém alguma definição antes de prosseguir
    if len(syn0) != 0:
      vals.clear()
      for wd1 in str1:
        syn1 = wn.synsets(wd1)
        # verifica se 2a palavra contém alguma definição antes de calcular similaridade
        if len(syn1) != 0:
          aux = syn0[0].wup_similarity(syn1[0])
          # se similaridade = 1, palavras iguais, break
          if aux == 1:
            break
          # senão, adiciona valor à lista vals
          else:
            vals.append(aux)
            #print("syn0:",syn0[0],"syn1:",syn1[0],"wup:",syn0[0].wup_similarity(syn1[0]))
      # se aux = 1, similaridade máxima = 1
      if aux == 1:
         means.append(aux)
      # senão seleciona máxima da lista
      else:
        means.append(max(vals))
  print(means)
  print(sum(means)/len(means))

str0 = filtertuples(preprocess(urls[0]),True)
str1 = filtertuples(preprocess(urls[1]),True)
str2 = filtertuples(preprocess(urls[2]),True)

print("string 0:",str0['word'].tolist())
print("string 1:",str1['word'].tolist())
similaridade_WP_means(str0['word'].tolist(),str1['word'].tolist())
similaridade_WP_max(str0['word'].tolist(),str1['word'].tolist())
dt = datamerge(str0,str1,'word','pos_tag')
print(dt)
jaccard_simples(dt['doc_freq'].tolist())

print('\n')
print("string 0:",str0['word'].tolist())
print("string 2:",str2['word'].tolist())
similaridade_WP_means(str0['word'].tolist(),str2['word'].tolist())
similaridade_WP_max(str0['word'].tolist(),str2['word'].tolist())
dt = datamerge(str0,str2,'word','pos_tag')
print(dt)
jaccard_simples(dt['doc_freq'].tolist())

# cosine do scikit
print('\ncosine do scikit')
corpus = [urls[0],urls[1]]
vectorizer = TfidfVectorizer()
trsfm=vectorizer.fit_transform(corpus)
print(cosine_similarity(trsfm[0:1], trsfm))
corpus = [urls[0],urls[2]]
vectorizer = TfidfVectorizer()
trsfm=vectorizer.fit_transform(corpus)
print(cosine_similarity(trsfm[0:1], trsfm))
corpus = [urls[0],urls[1],urls[2]]
vectorizer = TfidfVectorizer()
trsfm=vectorizer.fit_transform(corpus)
print(cosine_similarity(trsfm[0:1], trsfm))


string 0: ['Airbnb', 'shut', 'listings', 'China', 'two', 'years', 'lockdowns', 'country']
string 1: ['Starting', 'summer', 'Airbnb', 'take', 'listings', 'offers', 'hosted', 'experiences', 'China']
[0.17105672105672104, 0.29106753812636166, 0.1999273783587509, 0.21296296296296294, 0.25932150638032986, 0.26241529750301684, 0.19814814814814813]
0.22784279321947018
[0.25, 1.0, 1.0, 0.4, 0.7142857142857143, 0.631578947368421, 0.26666666666666666]
0.6089330469029717
           word pos_tag  freq_x  freq_y  doc_freq
0        Airbnb     NNP     2.0     1.0         2
3         China     NNP     2.0     1.0         2
8      Starting     VBG     0.0     1.0         1
7       country      NN     1.0     0.0         1
13  experiences     NNS     0.0     1.0         1
12       hosted      JJ     0.0     1.0         1
2      listings     NNS     2.0     1.0         2
6     lockdowns     NNS     1.0     0.0         1
11       offers     NNS     0.0     1.0         1
1          shut      VB     2.0    

In [ ]:
def getclasses(string):
  lemmas = []
  hypernyms = []
  na = 'N\A'
  for wd in string:
    syn = wn.synsets(wd)
    if len(syn) > 0:
      lemmas.append(syn[0].lemmas()[0].name())
      if len(syn[0].hypernyms()) > 0:
        hypernyms.append(syn[0].hypernyms()[0].name())
      else:
        hypernyms.append(na)
    else:
      lemmas.append(na)
      hypernyms.append(na)
  return pd.DataFrame(zip(string,lemmas,hypernyms),columns =['word','lemma','hypernym'])

def intersects(df0,df1,col):
  aux = []
  for i in df0.tolist():
    if i != 'N\A':
      aux.append(i)
  df0 = pd.DataFrame(zip(aux),columns =[col])
  df0['freq'] = df0.groupby([col])[col].transform('count')
  print(df0)
  aux.clear()
  for i in df1.tolist():
    if i != 'N\A':
      aux.append(i)
  df1 = pd.DataFrame(zip(aux),columns =[col])
  df1['freq'] = df1.groupby([col])[col].transform('count')
  print(df1)
  merged = pd.merge(df0,df1, on=[col], how='outer')
  merged = dataclean(merged).fillna(0)
  return merged

print(preprocess(urls[0]))
print(filtertuples(preprocess(urls[0]),False))
str0 = getclasses(filtertuples(preprocess(urls[0]),False))
print(str0,'\n')

print(preprocess(urls[1]))
print(filtertuples(preprocess(urls[1]),False))
str1 = getclasses(filtertuples(preprocess(urls[1]),False))
print(str1,'\n')

print(intersects(str0['lemma'],str1['lemma'],'lemma'))
print(intersects(str0['hypernym'],str1['hypernym'],'hypernym'))


[('Airbnb', 'NNP'), ('will', 'MD'), ('shut', 'VB'), ('down', 'RP'), ('its', 'PRP$'), ('listings', 'NNS'), ('in', 'IN'), ('China', 'NNP'), ('after', 'IN'), ('two', 'CD'), ('years', 'NNS'), ('of', 'IN'), ('lockdowns', 'NNS'), ('in', 'IN'), ('the', 'DT'), ('country', 'NN'), ('Airbnb', 'NNP'), ('will', 'MD'), ('shut', 'VB'), ('down', 'RP'), ('its', 'PRP$'), ('listings', 'NNS'), ('in', 'IN'), ('China', 'NNP')]
['Airbnb', 'shut', 'listings', 'China', 'two', 'years', 'lockdowns', 'country', 'Airbnb', 'shut', 'listings', 'China']
         word     lemma             hypernym
0      Airbnb       N\A                  N\A
1        shut     close                  N\A
2    listings      list        database.n.01
3       China     China                  N\A
4         two       two           digit.n.01
5       years   old_age    time_of_life.n.01
6   lockdowns  lockdown    imprisonment.n.03
7     country     state  political_unit.n.01
8      Airbnb       N\A                  N\A
9        shut     clos